## Upload data under data/ and hts_utils.py under utils/

This notebook was heavily modified from here:

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/NonNegativeReconciliation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install hierarchicalforecast statsforecast

In [2]:
import numpy as np
import pandas as pd

from utils.hts_eda_utils import *

from hierarchicalforecast.utils import HierarchicalPlot
from statsforecast.models import * # ARIMA, ETS, etc.
from statsforecast.core import StatsForecast

# TODO TopDown() reconciler causes KeyError 'ETS, Naive'
# TODO What happens with different reconc. methods? Is best one chosen? df doesn't seem to have extra columns
from hierarchicalforecast.methods import * # Reconcialiation methods: BottomUp, TopDown, MinTrace etc.
from hierarchicalforecast.core import HierarchicalReconciliation

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [33]:
# dataset subset to use? # Use full initially
#     deal_w_zeros_method = remove_zero_columns(df, any_or_all='any')

SELECT_TOP_K_PRODUCTS = None # None = keep all


# CHOOSE TIME SERIES METHODS HERE! https://nixtla.github.io/statsforecast/src/core/models_intro.html
TSModels = [
    ETS(season_length=7, model='ZAA'),
    Naive(),
    # AutoETS(season_length=7, model='ZAA'),
    # Naive(),
    # ARIMA(),
    # SeasonalExponentialSmoothingOptimized(season_length=7),
    # AutoRegressive(lags=6),
    # RandomWalkWithDrift()
    ]

# https://nixtla.github.io/hierarchicalforecast/methods.html
reconciliation_methods = [
    BottomUp(),
    TopDown(method='forecast_proportions'), # 'average_proportions' causes KeyError below
    MinTrace(method='ols'),
    OptimalCombination(method='ols'),
    ERM(method='closed') # Empirical Risk Minimization
]

TIME_SERIES_FREQ = 'M'
df = pd.read_excel('data/Quarterly_smoothing.xlsx', index_col=0)#.iloc[:,:5])



/usr/local/lib/python3.10/dist-packages/statsforecast/models.py:732: FutureWarning: `ETS` will be deprecated in future versions of `StatsForecast`. Please use `AutoETS` instead.
  ETS._warn()


In [4]:
dataset_hierarchy_delimiter = ' - ' # The delimiter currently used in the dataset
HIERARCHY_DELIMITER = '_' # '_' is needed by HierarchicalForecast. Need to replace

## 1. Load Data

<font color='blue'>Ariel: Replace ` - ` with `_`. I think `_` is used as Hierarchy level split by HF package (not sure)</font>

In [5]:
df.columns = df.columns.str.replace(' - ', HIERARCHY_DELIMITER) # Replace Hierarchy delimiter

##### Columns of all zeros cause errors (Division by zero in Covariance calc.). Need to fix

In [6]:
# TODO make this transform a parameter too
df = add_1_to_all_df_cells(df)

df.head()

,Дальневосточный ФО_ADRIANOL_Adrianol for adults nasal drops 10 ml #1,Дальневосточный ФО_AGALATES_Agalates tabs 0.5 mg #2,Дальневосточный ФО_AGALATES_Agalates tabs 0.5 mg #8,Дальневосточный ФО_ALMAGEL_Almagel A susp 170 ml #1,Дальневосточный ФО_ALMAGEL_Almagel Neo sachet 10 ml #10,Дальневосточный ФО_ALMAGEL_Almagel Neo susp 170 ml #1,Дальневосточный ФО_ALMAGEL_Almagel sachet 10 ml #10,Дальневосточный ФО_ALMAGEL_Almagel susp 170 ml #1,Дальневосточный ФО_ALMONT_Almont FC tabs 10 mg #28,Дальневосточный ФО_ALMONT_Almont chew tabs 4 mg #28,...,Южный ФО_VELBINE_Velbine solution for inf 10 mg/ml 5ml #1,Южный ФО_VESTIBO_Vestibo tabs 16 mg #30,Южный ФО_VESTIBO_Vestibo tabs 24 mg #30,Южный ФО_VINCRISTINE-TEVA_Vincristine-Teva lyoph for inf 1 mg/ml 1 ml #1,Южный ФО_VINCRISTINE-TEVA_Vincristine-Teva lyoph for inf 1 mg/ml 2 ml #1,Южный ФО_VINORELBINE-TEVA_VINORELBIN-TEVA 50 mg.5 ml,Южный ФО_VINORELBINE-TEVA_VINORELBINE-TEVA concentrate 10 mg.ml 1 ml,Южный ФО_ZINCTERAL_Zincteral-Teva FC tabs 124 mg #150,Южный ФО_ZINCTERAL_Zincteral-Teva FC tabs 124 mg #25,Южный ФО_ZOLEDRONAT-TEVA_Zoledronate-Teva concentrate for inf 4 mg/5ml 5 ml #1
Month,,,,,,,,,,,,,,,,,,,,,
2018-03-01,200,1,1,10,1,1,1,949,36,88,...,106,1,315,401,1,1,1,1,17,9690
2018-04-01,1000,1,1,10,1,1,1,1037,36,88,...,1046,1,248,2161,1,1,1,1,1,11705
2018-05-01,1731,1,1,302,1,1,1,1246,94,148,...,1011,1,248,3246,1,1,1,1,1,15233
2018-06-01,2090,1,1,491,1,1,1,1787,184,104,...,947,1,1,3236,1,1,1,1,1,6586
2018-07-01,1547,1,61,491,1,1,1,4132,184,104,...,46,1,1,5086,1,1,1,7,1,6035


##### Optional: Select only top Products

Saves compute

In [7]:
if SELECT_TOP_K_PRODUCTS is not None:
    df = select_top_n_brands(df, n=SELECT_TOP_K_PRODUCTS)

# df.head(5)

In [8]:
%%capture
df_with_aggregates, hierarchy = prep_data_for_scikit_hts(df)

df_with_aggregates.head(5)

<font color='cyan'>HierarchicalForecast likes data to be Drug | Date | Sales, rather than having DrugName as columns</font>


### Melt data into format required by HierarchicalForecast

Following how their example code's data looks

In [9]:
# Melt the DataFrame - convert ColNames to rows to match input to HierForecast
df_with_aggregates.reset_index(inplace=True) # Move Month index to column (package requirement)

# TODO Check these for prediction error
melted_df = df_with_aggregates.melt(id_vars=['Month'], var_name='Drug', value_name='Sales')

# Convert melted DataFrame to the required format
melted_df = melted_df[['Drug', 'Month', 'Sales']]

# Col names seem to need to be thus for package
melted_df.rename(columns={'Drug': 'unique_id', 'Month':'ds', 'Sales':'y'}, inplace=True)


print(melted_df.head())
print(melted_df.tail())


                                           unique_id         ds     y
0  Дальневосточный ФО_ADRIANOL_Adrianol for adult... 2018-03-01   200
1  Дальневосточный ФО_ADRIANOL_Adrianol for adult... 2018-04-01  1000
2  Дальневосточный ФО_ADRIANOL_Adrianol for adult... 2018-05-01  1731
3  Дальневосточный ФО_ADRIANOL_Adrianol for adult... 2018-06-01  2090
4  Дальневосточный ФО_ADRIANOL_Adrianol for adult... 2018-07-01  1547
       unique_id         ds         y
190489     Total 2022-07-01  49450935
190490     Total 2022-08-01  52831983
190491     Total 2022-09-01  52573920
190492     Total 2022-10-01  61896441
190493     Total 2022-11-01  68698749


## Creating `S_df`

All colored font is Ariel

<font color='turquoise'>We've created `Y_df, tags`. All we need is `S_df`</font>
This is like a tree representing the hierarchy, with aggregations at each level

<font color='blue'>`S_df` is a representation of the Hierarchy - 1 means that column name (item, Drugs in our case), belongs to the Total row. Rows represent totals at each level of the hierarchy, for each node</font>

In [10]:
S_df = create_S_df(df)

S_df

,Дальневосточный ФО_ADRIANOL_Adrianol for adults nasal drops 10 ml #1,Дальневосточный ФО_AGALATES_Agalates tabs 0.5 mg #2,Дальневосточный ФО_AGALATES_Agalates tabs 0.5 mg #8,Дальневосточный ФО_ALMAGEL_Almagel A susp 170 ml #1,Дальневосточный ФО_ALMAGEL_Almagel Neo sachet 10 ml #10,Дальневосточный ФО_ALMAGEL_Almagel Neo susp 170 ml #1,Дальневосточный ФО_ALMAGEL_Almagel sachet 10 ml #10,Дальневосточный ФО_ALMAGEL_Almagel susp 170 ml #1,Дальневосточный ФО_ALMONT_Almont FC tabs 10 mg #28,Дальневосточный ФО_ALMONT_Almont chew tabs 4 mg #28,...,Южный ФО_VELBINE_Velbine solution for inf 10 mg/ml 5ml #1,Южный ФО_VESTIBO_Vestibo tabs 16 mg #30,Южный ФО_VESTIBO_Vestibo tabs 24 mg #30,Южный ФО_VINCRISTINE-TEVA_Vincristine-Teva lyoph for inf 1 mg/ml 1 ml #1,Южный ФО_VINCRISTINE-TEVA_Vincristine-Teva lyoph for inf 1 mg/ml 2 ml #1,Южный ФО_VINORELBINE-TEVA_VINORELBIN-TEVA 50 mg.5 ml,Южный ФО_VINORELBINE-TEVA_VINORELBINE-TEVA concentrate 10 mg.ml 1 ml,Южный ФО_ZINCTERAL_Zincteral-Teva FC tabs 124 mg #150,Южный ФО_ZINCTERAL_Zincteral-Teva FC tabs 124 mg #25,Южный ФО_ZOLEDRONAT-TEVA_Zoledronate-Teva concentrate for inf 4 mg/5ml 5 ml #1
Total,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Дальневосточный ФО,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Дальневосточный ФО_ADRIANOL,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Дальневосточный ФО_ADRIANOL_Adrianol for adults nasal drops 10 ml #1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Дальневосточный ФО_AGALATES,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Южный ФО_ZINCTERAL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
Южный ФО_ZINCTERAL_Zincteral-Teva FC tabs 124 mg #150,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
Южный ФО_ZINCTERAL_Zincteral-Teva FC tabs 124 mg #25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Южный ФО_ZOLEDRONAT-TEVA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Create `tags`, which is a description of the Hierarchy as `dict`

Original `tags` loaded from example Dataset - they didn't create it programmatically

In [11]:
# hierarchy

In [12]:
# TODO wtf did chatgpt do here?
transformed_data = { # Need names for hierarchy levels IMO
    "Sales": ["Total"],
    "Sales/Region": hierarchy['Total'],
    "Sales/Region/DrugName": sum([hierarchy[region] for region in hierarchy['Total']], []),
    "Sales/Region/DrugName/DrugDosage": sum([hierarchy[key] for key in sum([hierarchy[region] for region in hierarchy['Total']], [])], []),
}

# Convert the lists to numpy arrays for consistency with the format
for key in transformed_data:
    transformed_data[key] = np.array(transformed_data[key], dtype=object)

# print(transformed_data)
tags = transformed_data

We split the dataframe in train/test splits.

In [13]:
Y_df = melted_df

# Y_df

In [14]:
Y_test_df = Y_df.groupby('unique_id').tail(7) # Original code
Y_train_df = Y_df.drop(Y_test_df.index)

Y_test_df = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

In [15]:
print(Y_test_df.head())
print(Y_test_df.tail())

                                                           ds  y
unique_id                                                       
Дальневосточный ФО_ADRIANOL_Adrianol for adults... 2022-05-01  0
Дальневосточный ФО_ADRIANOL_Adrianol for adults... 2022-06-01  0
Дальневосточный ФО_ADRIANOL_Adrianol for adults... 2022-07-01  0
Дальневосточный ФО_ADRIANOL_Adrianol for adults... 2022-08-01  0
Дальневосточный ФО_ADRIANOL_Adrianol for adults... 2022-09-01  0
                  ds         y
unique_id                     
Total     2022-07-01  49450935
Total     2022-08-01  52831983
Total     2022-09-01  52573920
Total     2022-10-01  61896441
Total     2022-11-01  68698749


## 2. Base Forecasts

The following cell computes the *base forecast* for each time series using the `ETS` and `naive` models. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [16]:
%%capture
fcst = StatsForecast(
    df=Y_train_df,
    # models=TSModels,
    models=[ETS(season_length=7, model='ZZA'), Naive()],
    freq=TIME_SERIES_FREQ,
    n_jobs=-1
)
Y_hat_df = fcst.forecast(h=7) # TODO What is h=7?

In [17]:
TSModels

[ETS, Naive]

In [18]:
[ETS(season_length=7, model='ZZA'), Naive()]# == TSModels

/usr/local/lib/python3.10/dist-packages/statsforecast/models.py:732: FutureWarning: `ETS` will be deprecated in future versions of `StatsForecast`. Please use `AutoETS` instead.
  ETS._warn()


[ETS, Naive]

Observe that the ETS model computes negative forecasts for some series.

<font color='pink'>Does `Y_hat_df` have a `ds` column in the original code?</font>

Yes

In [19]:
# Make sure S_df does not have extra entries - TODO I still don't know exactly how S_df is created
#   This is jerry-rigged to work
rows_to_drop = list(set(S_df.index) - set(Y_test_df.index))
# rows_to_drop
S_df.drop(rows_to_drop, inplace=True)

In [20]:
# `S_df` should have 1 entry for each unique row in `Y_hat_df`
assert(len(S_df.index) == len(set(Y_hat_df.index)))
assert(set(Y_train_df.index) - set(S_df.index) == set())
assert(set(S_df.index) - set(Y_train_df.index) == set())

## 3. Non-Negative Reconciliation

The following cell makes the previous forecasts coherent and nonnegative using the `HierarchicalReconciliation` class.

In [36]:
# reconcilers = [
#     BottomUp(),
#     # TopDown(method='forecast_proportions'),
#     # MiddleOut(middle_level='Country/Purpose/State', # ValueError here
#             #   top_down_method='forecast_proportions')
# ]
hrec = HierarchicalReconciliation(reconcilers=[TopDown(method='forecast_proportions')]) # 'average_proportions' causes KeyError below
    # MinTrace(method='ols'),
    # OptimalCombination(method='ols'),
    # ERM(method='closed'))
# TODO TopDown() reconciler causes KeyError 'ETS, Naive'

Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df,
                          S=S_df, tags=tags)

/usr/local/lib/python3.10/dist-packages/hierarchicalforecast/methods.py:261: RuntimeWarning: invalid value encountered in divide
  reconciled[idx_child] = y_hat[idx_child] * fcst_parent / childs_sum


Observe that the nonnegative reconciliation method obtains nonnegative forecasts.

## 4. Evaluation

The `HierarchicalForecast` package includes the `HierarchicalEvaluation` class to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [ ]:
from hierarchicalforecast.evaluation import HierarchicalEvaluation

In [ ]:
# TODO enhance this
def mse(y, y_hat):
    return np.mean((y-y_hat)**2)

evaluator = HierarchicalEvaluation(evaluators=[mse])
evaluation = evaluator.evaluate(
        Y_hat_df=Y_rec_df, Y_test_df=Y_test_df,
        tags=tags, benchmark='Naive'
)
evaluation.filter(like='ETS', axis=1).T

Observe that the nonnegative reconciliation method performs better that its unconstrained counterpart.

## Plot Hierarchy & Evaluations

In [ ]:
a = Y_test_df.sort_index()#.sort_values(by='ds', ascending=True)

# TODO programmatically get these by subtracting column names (set)
b = Y_rec_df[['ETS', 'Naive', 'ETS/BottomUp', 'Naive/BottomUp']].sort_index()#.sort_values(by='ds', ascending=True)#.drop(columns=['ds'])

b

In [ ]:
merged_test_preds_df = pd.concat([a, b], axis=1)
# merged_test_preds_df

In [ ]:
merged_test_preds_df = merged_test_preds_df.sort_values(by='ds', ascending=True)
merged_test_preds_df

In [ ]:
hplt = HierarchicalPlot(S=S_df, tags=tags)

hplt.plot_hierarchical_predictions_gap(Y_df=merged_test_preds_df, models = 'ETS')#['ETS', 'Naive', 'ETS/BottomUp', 'Naive/BottomUp'])

In [ ]:
hplt.plot_hierarchically_linked_series(bottom_series='Южный ФО_VALZ_Valz N FC tabs 80 mg/12.5mg #28', Y_df=Y_train_df)

### References
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition:
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)
- [Wickramasuriya, S. L., Athanasopoulos, G., & Hyndman, R. J. (2019). \"Optimal forecast reconciliation for
    hierarchical and grouped time series through trace minimization\". Journal of the American Statistical Association,
    114 , 804–819. doi:10.1080/01621459.2018.1448825.](https://robjhyndman.com/publications/mint/).
- [Wickramasuriya, S.L., Turlach, B.A. & Hyndman, R.J. (2020). \"Optimal non-negative
    forecast reconciliation". Stat Comput 30, 1167–1182,
    https://doi.org/10.1007/s11222-020-09930-0](https://robjhyndman.com/publications/nnmint/).